In [6]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp

In [2]:
df = pd.read_csv("experimento_teste_ab.csv")

### 1. Qual dos cenários tem a maior taxa de conversão?

In [3]:
# Calcular a taxa de conversão para a versão A
total_conversoes_A = df[df['Versão_Página'] == 'A']['Conversões'].sum()
total_visualizacoes_A = df[df['Versão_Página'] == 'A']['Visualizações'].sum()
taxa_conversao_A = (total_conversoes_A / total_visualizacoes_A) * 100

# Calcular a taxa de conversão para a versão B
total_conversoes_B = df[df['Versão_Página'] == 'B']['Conversões'].sum()
total_visualizacoes_B = df[df['Versão_Página'] == 'B']['Visualizações'].sum()
taxa_conversao_B = (total_conversoes_B / total_visualizacoes_B) * 100

print("Taxa de conversão A:", taxa_conversao_A, "%")
print("Taxa de conversão B:", taxa_conversao_B, "%")

Taxa de conversão A: 0.06961249046971857 %
Taxa de conversão B: 0.08295946268815206 %


### 2. Considerando o cenário A como o inicial, onde queremos um aumento para 10% de conversão. Qual o valor de Delta (diferença entre médias ou proporções das métricas dos dois grupos)?

In [ ]:
taxa_conversao_alvo = 0.10  # 10%

delta = taxa_conversao_alvo - taxa_conversao_A

print("Delta:", delta)

Delta: 0.030387509530281437


### 3. Calcule a variância amostral do cenário A, considerado o inicial (de controle).



In [ ]:
taxa_conversao_A = df[df['Versão_Página'] == 'A']['Conversões'].sum() / df[df['Versão_Página'] == 'A']['Visualizações'].sum()

# Calcular a variável binomial para cada observação no grupo controle
observacoes_controle = df[df['Versão_Página'] == 'A']['Conversões'].tolist()

# Calcular a variância amostral do grupo controle
variancia_amostral_A = np.var(observacoes_controle)

print("Variância amostral do grupo controle (S1):", variancia_amostral_A)

Variância amostral do grupo controle (S1): 0.09397499999999999


### 4. Calcule qual o tamanho da amostra necessária para o desenvolvimento de um teste A/B, seguindo os seguintes critérios:
* O cenário A, da base, como o inicial, que funciona hoje.
* Considere 95% de confiança de que o efeito na conversão não foi aleatório com um nível de significância de 5% (alpha = 0,05).
* Também considere 80% de certeza conseguir capturar o efeito da nova abordagem.
* O aumento para 10% de conversão.



In [ ]:
import scipy.stats as stats

alpha = 0.05
beta = 1 - 0.80

z_alpha_2 = stats.norm.ppf(1 - alpha/2)

z_beta = stats.norm.ppf(1 - beta)

# Calculando o tamanho da amostra necessário
n = (2 * (variancia_amostral_A) * (z_alpha_2 + z_beta)**2) / delta**2

print("Tamanho da amostra necessário em cada grupo:", round(n))


Tamanho da amostra necessário em cada grupo: 1598


In [32]:
pacientes = pd.read_csv("pacientes.csv", index_col='Unnamed: 0')

#### 3. Considerando uma amostra de 45 números ([909, 751, 402, 400, 726, 39, 184, 269, 255, 769, 209, 715, 677, 381, 793, 697,  89, 280, 232, 756, 358,  36, 439, 768, 967, 699, 473, 222,  89, 639, 883, 558, 757,  84, 907, 895, 217, 224, 311, 348, 146, 505, 273, 957, 362]). Considerando essa amostra é possível dizer que a idade média das pessoas com problemas cardíacos é maior que 50 anos? Nível de significância igual a 5%.


In [8]:
indices_amostra = [909, 751, 402, 400, 726, 39, 184, 269, 255, 769, 209, 715, 677, 381, 793, 697, 89, 280, 232, 756, 358, 36, 439, 768, 967, 699, 473, 222, 89, 639, 883, 558, 757, 84, 907, 895, 217, 224, 311, 348, 146, 505, 273, 957, 362]

amostra_idades = pacientes.loc[indices_amostra]['Idade']

_, p_value = ttest_1samp(amostra_idades, 50)

alpha = 0.05

if p_value < alpha:
    print("Rejeitamos a hipótese nula.")
else:
    print("Não rejeitamos a hipótese nula.")

Não rejeitamos a hipótese nula.


#### 5. Agora considere o um conjunto de pessoas aleatória que representam o index do dataframe,  índices = ([690, 894,  67, 201, 364,  19,  60, 319, 588, 643, 855, 623, 530, 174, 105, 693,   6, 462, 973, 607, 811, 346, 354, 966, 943, 372]), podemos dizer que a pressão arterial média para pacientes com condições de saúde adicionais é igual à pressão arterial média para pacientes sem condições adicionais de saúde? Considere o nível de significância a 6%.

In [18]:
from scipy.stats import ttest_ind

indices_com_condicoes = [690, 894, 67, 201, 364, 19, 60, 319, 588, 643, 855, 623, 530, 174, 105, 693, 6, 462, 973, 607, 811, 346, 354, 966, 943, 372]
parcial = pacientes.loc[indices_com_condicoes]

pressao_com_condicoes_adicionais = parcial[parcial['Nome_Estado_Saude'] != 'Saudável']['Pressao_Arterial']
pressao_sem_condicoes_adicionais = parcial[parcial['Nome_Estado_Saude'] == 'Saudável']['Pressao_Arterial']

statistic, p_value = ttest_ind(pressao_com_condicoes_adicionais, pressao_sem_condicoes_adicionais)

alpha = 0.06

if p_value < alpha:
    print("Rejeitamos a hipótese nula.")
else:
    print("Não rejeitamos a hipótese nula.")

Não rejeitamos a hipótese nula.


#### 6. Existe uma diferença significativa na pressão arterial média entre diferentes grupos étnicos nesta população? (Teste ANOVA, alpha é 5%)
*  Hipótese Nula (H0): A pressão arterial média é a mesma em todos os grupos étnicos.
*  Hipótese Alternativa (H1): Há uma diferença significativa na pressão arterial média entre pelo menos dois grupos étnicos nesta população.


In [39]:
from scipy.stats import f_oneway

pressao_por_etnia = {
    '1': pacientes[pacientes['Nome_Etnia'] == 'Afro-americano']['Pressao_Arterial'].values,
    '2': pacientes[pacientes['Nome_Etnia'] == 'Asiático']['Pressao_Arterial'].values,
    '3': pacientes[pacientes['Nome_Etnia'] == 'Hispânico']['Pressao_Arterial'].values,
    '4': pacientes[pacientes['Nome_Etnia'] == 'Caucasiano']['Pressao_Arterial'].values,

}

statistic, p_value = f_oneway(*pressao_por_etnia.values())

alpha = 0.05

if p_value < alpha:
    print("Rejeitamos a hipótese nula.")
else:
    print("Não rejeitamos a hipótese nula.")


Não rejeitamos a hipótese nula.


#### 7. Lúcia é uma pesquisadora e tem o objetivo de entender a relação de gênero neste grupo de pacientes. Acredita-se que há uma relação entre o sexo e condições de saúde adicionais. (Teste qui-quadrado)


In [70]:
frequencia_condicoes_adicionais_para_homem = pacientes[(pacientes['Nome_Estado_Saude']!='Saudável') & (pacientes['Nome_Genero'] == 'Masculino')].shape[0]
frequencia_sem_condicoes_adicionais_para_homem = pacientes[(pacientes['Nome_Estado_Saude']=='Saudável') & (pacientes['Nome_Genero'] == 'Masculino')].shape[0]
frequencia_condicoes_adicionais_para_mulher = pacientes[(pacientes['Nome_Estado_Saude']!='Saudável') & (pacientes['Nome_Genero'] == 'Feminino')].shape[0]
frequencia_sem_condicoes_adicionais_para_mulher = pacientes[(pacientes['Nome_Estado_Saude']=='Saudável') & (pacientes['Nome_Genero'] == 'Feminino')].shape[0]


In [72]:
from scipy.stats import chi2_contingency

tabela_contingencia = [
    [frequencia_condicoes_adicionais_para_homem, frequencia_sem_condicoes_adicionais_para_homem],
    [frequencia_condicoes_adicionais_para_mulher, frequencia_sem_condicoes_adicionais_para_mulher]
]

statistic, p_value, dof, expected = chi2_contingency(tabela_contingencia)

alpha = 0.05

if p_value < alpha:
    print("Rejeitamos a hipótese nula.")
else:
    print("Não rejeitamos a hipótese nula.")


Não rejeitamos a hipótese nula.


#### 8. Existe uma associação entre a idade dos pacientes e sua pressão arterial?
*  Hipótese Nula (H0): A idade dos pacientes é independente da pressão arterial.
*  Hipótese Alternativa (H1): A idade dos pacientes está associada à sua pressão arterial.


In [85]:
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

idades = pacientes.Idade
pressoes_arteriais = pacientes.Pressao_Arterial

faixas_etarias = [30, 40, 50, 60, 70, 80]
intervalos_pressao = [100, 120, 140, 160, 180, 200]

# Contar o número de pacientes em cada faixa etária e intervalo de pressão arterial
contagem_idades = np.histogram(idades, bins=faixas_etarias)[0]
contagem_pressao = np.histogram(pressoes_arteriais, bins=intervalos_pressao)[0]

# Criar uma tabela de contingência
tabela_contingencia = np.array([contagem_idades, contagem_pressao])

# Realizar o teste do qui-quadrado de independência
statistic, p_value, dof, expected = chi2_contingency(tabela_contingencia)

alpha = 0.05

# Interpretar os resultados
if p_value < alpha:
    print("Rejeitamos a hipótese nula.")
else:
    print("Não rejeitamos a hipótese nula.")


Rejeitamos a hipótese nula.


In [79]:
pacientes.describe()

,Pressao_Arterial,Idade,Genero,Etnia,Estado_Saude
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
mean,150.375623,49.757000,0.503000,1.526000,0.51000
std,9.943613,11.682998,0.500241,1.104783,0.50015
min,112.929821,30.000000,0.000000,0.000000,0.00000
25%,144.240948,40.000000,0.000000,1.000000,0.00000
50%,150.455624,49.000000,1.000000,2.000000,1.00000
75%,156.792113,60.000000,1.000000,3.000000,1.00000
max,181.803131,70.000000,1.000000,3.000000,1.00000


#### 9. Qual é o intervalo de confiança para a média da pressão arterial entre os pacientes com condições de saúde adicionais? (nível de confiança 95%)


In [91]:
from scipy.stats import norm

pressoes_arteriais_condicoes = pacientes.Pressao_Arterial

media_pressao_condicoes = np.mean(pressoes_arteriais_condicoes)
desvio_padrao_pressao_condicoes = np.std(pressoes_arteriais_condicoes)

n = len(pressoes_arteriais_condicoes)

nivel_confianca = 0.95

z = norm.ppf((1 + nivel_confianca) / 2)

# Calcular o intervalo de confiança
intervalo_confianca = z * (desvio_padrao_pressao_condicoes / np.sqrt(n))

limite_inferior = media_pressao_condicoes - intervalo_confianca
limite_superior = media_pressao_condicoes + intervalo_confianca

print("Limite Inferior:", limite_inferior)
print("Limite Superior:", limite_superior)


Limite Inferior: 149.75963067457386
Limite Superior: 150.9916145564712


#### 10. A distribuição da pressão arterial na população segue uma distribuição normal?
*  Hipótese Nula (H0): A distribuição da pressão arterial na população segue uma distribuição normal.
*  Hipótese Alternativa (H1): A distribuição da pressão arterial na população não segue uma distribuição normal.


In [94]:
from scipy.stats import kstest, norm

pressao_arterial_populacao =  pacientes.Pressao_Arterial

# Realizar o teste de Kolmogorov-Smirnov
statistic, p_value = kstest(pressao_arterial_populacao, 'norm')

alpha = 0.05

if p_value < alpha:
    print("Rejeitamos a hipótese nula.")
else:
    print("Não rejeitamos a hipótese nula.")


Rejeitamos a hipótese nula.
